In [1]:
from npsurvival.FeatureEngineer import FeatureEngineer
from npsurvival.Utils import model_prepare


Using TensorFlow backend.


In [19]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import re


In [3]:
import rpy2.robjects as robjects
from rpy2.robjects import FloatVector, IntVector
from rpy2.robjects.packages import importr
survival = importr("survival")
base = importr('base')


In [8]:
fe = FeatureEngineer(verbose=False, data_path="../../dataset/")
sources = fe.get_diseases_list()

dataset_idx = 0
file_prefix = ""
low_event_thd = 0.03
low_value_thd = low_event_thd / 3
patient_dict, feature_set, train_id_list, test_id_list = \
    fe.load_data_as_dict(dataset_idx, file_prefix=file_prefix, 
                         low_freq_event_thd=low_event_thd, 
                         low_freq_value_thd=low_value_thd)
train_df, test_df, feature_list = \
    model_prepare(patient_dict, feature_set, train_id_list, test_id_list)


/usr/local/lib/python3.6/site-packages/fancyimpute/soft_impute.py:100: RuntimeWarning: divide by zero encountered in double_scalars
  return (np.sqrt(ssd) / old_norm) < self.convergence_threshold


In [83]:
def get_r_var_name(index, column):
#     return re.sub(r"[:/_.()#><=,\-+*%'\?]+", ".", column)
    return "C" + str(index)


In [143]:
train_x = train_df.drop(columns=["LOS", "OUT"]).values

standardalizer = StandardScaler()
standardalizer.fit(train_x)
train_x = standardalizer.transform(train_x)

pca = PCA(n_components=17)
pca.fit(train_x)
reduced_x = pca.transform(train_x)
reduced_train_df = pd.DataFrame(reduced_x).set_index(train_df.index)
columns = ["C" + str(i) for i in range(reduced_train_df.shape[1])]
reduced_train_df.columns = columns
reduced_train_df["LOS"] = train_df["LOS"]
reduced_train_df["OUT"] = train_df["OUT"]
reduced_train_df.head()


,C0,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,LOS,OUT
1030.0,44.194034,-13.397990,11.534903,11.687051,-1.318688,1.104402,-5.640882,-0.031946,-3.630892,-1.240038,-1.100930,-7.232711,-1.332783,0.076291,-0.250560,4.479930,0.641895,3.429572,1
82960.0,23.498330,38.475077,-11.258959,14.749113,-4.716190,-1.138436,-2.163540,-5.344575,2.149270,2.070083,1.149682,-1.398284,0.916809,-1.303685,-0.301335,-1.598796,-0.751293,2.721134,1
77842.0,42.536659,13.534693,12.970034,1.931330,14.014961,-16.366585,9.851383,-0.080984,1.308945,0.696421,8.097058,-2.172486,-3.592421,4.384633,2.141050,0.334008,-0.622001,32.856181,1
15379.0,-35.761768,-6.058173,0.061588,5.709436,-0.812907,-1.125814,-0.188029,1.805742,1.347554,-0.294843,0.914754,0.643538,0.329827,0.124009,-0.036260,-0.376276,0.079378,0.981250,1
81938.0,-10.154029,1.242907,0.889880,-3.437245,1.511405,0.048490,-1.097172,-0.610982,-1.930113,-1.327312,-1.432094,-0.580100,0.720366,-0.243927,-0.383972,-1.290887,-0.480280,2.917454,1


In [144]:
test_x = test_df.drop(columns=["LOS", "OUT"]).values
test_x = standardalizer.transform(test_x)
reduced_x = pca.transform(test_x)
reduced_test_df = pd.DataFrame(reduced_x).set_index(test_df.index)
columns = ["C" + str(i) for i in range(reduced_test_df.shape[1])]
reduced_test_df.columns = columns
reduced_test_df["LOS"] = test_df["LOS"]
reduced_test_df["OUT"] = test_df["OUT"]


In [145]:
column_ref = {}
df_command = 'train.df <- data.frame('
sr_command = 'survregWeibull <- survreg(s ~ '
for index, column in enumerate(reduced_train_df.drop(columns=["LOS", "OUT"]).columns):
    r_var_name = get_r_var_name(index, column)
    column_ref[column] = r_var_name
    robjects.globalenv[r_var_name] = FloatVector(list(reduced_train_df[column]))
    df_command += r_var_name + ', '
    sr_command += r_var_name
    if index != len(reduced_train_df.columns) - 3:
        sr_command += ' + '
robjects.globalenv["LOS"] = FloatVector(list(reduced_train_df["LOS"]))
robjects.globalenv["OUT"] = FloatVector(list(reduced_train_df["OUT"]))
df_command += 'LOS, OUT)'
sr_command += ', train.df, dist = "weibull")'
robjects.r(df_command)
robjects.r('s <- Surv(train.df$LOS, train.df$OUT)')
robjects.r(sr_command)


R object with classes: ('survreg',) mapped to:
<ListVector - Python:0x11d4e7e48 / R:0x7ffb10528c98>
[FloatVector, FloatVector, Matrix, FloatVector, ..., FloatVector, Vector, StrVector, Matrix]
  coefficients: <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x111517688 / R:0x7ffb22f50cb0>
[1.841309, -0.001066, 0.009792, 0.026685, ..., -0.130009, 0.008769, -0.002161, -0.025374]
  icoef: <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x11e33b888 / R:0x7ffb2c5aa988>
[1.881371, 0.199635]
  var: <class 'rpy2.robjects.vectors.Matrix'>
  R object with classes: ('matrix',) mapped to:
<Matrix - Python:0x11f648a48 / R:0x7ffb2f1e7800>
[0.005056, -0.000000, 0.000023, -0.000003, ..., 0.000012, 0.000036, -0.000002, 0.001912]
  loglik: <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x11e2d4808 / R:0x7ffb2c5aab48>
[-829.027492, -807.106559]
...
  linear.predictors: <class 'rpy2.robjects.vectors.FloatVector'>
  R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x11e2d4908 / R:0x7ffb22f525a0>
[1.000000, 0.000000, 0.000000, -0.000000, ..., -0.000000, 0.000000, 0.000000, 0.000000]
  df: <class 'rpy2.robjects.vectors.Vector'>
  R object with classes: ('survreg',) mapped to:
<Vector - Python:0x11d4a7cc8 / R:0x7ffb2c8c9438>
[RObject, Vector, RObject, StrVector]
  scale: <class 'rpy2.robjects.vectors.StrVector'>
  R object with classes: ('character',) mapped to:
<StrVector - Python:0x11e2d42c8 / R:0x7ffb1062de40>
['weibull']
  idf: <class 'rpy2.robjects.vectors.Matrix'>
  R object with classes: ('matrix',) mapped to:
<Matrix - Python:0x11def7108 / R:0x7ffb2f22ee00>
[1.232435, 1.001049, 3.492140, -0.018928, ..., 1.000000, 1.000000, 1.000000, 1.000000]

In [155]:
robjects.r('pct <- seq(.01,.99,by=.01)')
pred_median = []
true_median = reduced_test_df["LOS"]
for _, row in reduced_test_df.iterrows():
    pd_command = 'result <- predict(survregWeibull, newdata=list('
    for index, column in enumerate(column_ref):
        r_var_name = column_ref[column]
        pd_command += r_var_name + '=' + str(row[column])
        if index != len(column_ref) - 1:
            pd_command += ','
        else:
            pd_command += '), type="quantile", p=pct)'
    result = list(robjects.r(pd_command))
    pred_median.append(result[49])


In [156]:
from npsurvival.Utils import evaluate_predict_result
evaluate_predict_result(pred_median, reduced_test_df, print_result=True, show_time=True)


concordance: 0.615145723841376
----------
pred: 4.06333330210005 ; true: 6.950532407407407
pred: 2.9042149861344666 ; true: 3.878483796296296
pred: 2.75363069838316 ; true: 5.873425925925926
pred: 1.203674501174483 ; true: 0.9871296296296296
pred: 6.579701937280747 ; true: 2.763449074074074
pred: 4.923891191352622 ; true: 1.5623842592592592
pred: 2.3249928599908873 ; true: 0.05664351851851852
pred: 4.427889746722779 ; true: 0.6035879629629629
pred: 5.641860751783552 ; true: 4.758472222222222
pred: 2.7377861630987566 ; true: 1.1085532407407408
pred: 5.700388867379193 ; true: 2.9381944444444446
pred: 4.3762262947770845 ; true: 4.974699074074074
pred: 3.330013284202248 ; true: 4.601898148148148
pred: 5.157915038066493 ; true: 13.967731481481481
pred: 4.575425601153157 ; true: 1.9834722222222223
pred: 6.969041259767759 ; true: 4.115277777777778
pred: 4.14721282050908 ; true: 4.581909722222222
pred: 3.508239119851253 ; true: 0.7576388888888889
pred: 3.4877736748187 ; true: 14.64947916666666

0.615145723841376